In [210]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split

# Display preferences
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore")

In [211]:
# Load the houseprices data from Thinkful's database.
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices_df = pd.read_sql_query('select * from houseprices',con=engine)
houseprices_df.head(2)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500


In [212]:
houseprices_df_new=houseprices_df.dropna(axis=1,how='any')

In [213]:
houseprices_df_new.drop('id',axis=1,inplace=True)

In [214]:
houseprices_df_new.select_dtypes('object').columns

Index(['mszoning', 'street', 'lotshape', 'landcontour', 'utilities',
       'lotconfig', 'landslope', 'neighborhood', 'condition1', 'condition2',
       'bldgtype', 'housestyle', 'roofstyle', 'roofmatl', 'exterior1st',
       'exterior2nd', 'exterqual', 'extercond', 'foundation', 'heating',
       'heatingqc', 'centralair', 'kitchenqual', 'functional', 'paveddrive',
       'saletype', 'salecondition'],
      dtype='object')

In [215]:
houseprices_df_new.select_dtypes(['int64','float64']).columns

Index(['mssubclass', 'lotarea', 'overallqual', 'overallcond', 'yearbuilt',
       'yearremodadd', 'bsmtfinsf1', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf',
       'firstflrsf', 'secondflrsf', 'lowqualfinsf', 'grlivarea',
       'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath', 'bedroomabvgr',
       'kitchenabvgr', 'totrmsabvgrd', 'fireplaces', 'garagecars',
       'garagearea', 'wooddecksf', 'openporchsf', 'enclosedporch',
       'threessnporch', 'screenporch', 'poolarea', 'miscval', 'mosold',
       'yrsold', 'saleprice'],
      dtype='object')

In [216]:
cat_df=houseprices_df[['mszoning', 'street', 'lotshape', 'landcontour', 'utilities',
       'lotconfig', 'landslope', 'neighborhood', 'condition1', 'condition2',
       'bldgtype', 'housestyle', 'roofstyle', 'roofmatl', 'exterior1st',
       'exterior2nd', 'exterqual', 'extercond', 'foundation', 'heating',
       'heatingqc', 'centralair', 'kitchenqual', 'functional', 'paveddrive',
       'saletype', 'salecondition']]
cat_df_enc=pd.get_dummies(cat_df,drop_first=True)

In [217]:
num_df=houseprices_df[['mssubclass', 'lotarea', 'overallqual', 'overallcond', 'yearbuilt',
       'yearremodadd', 'bsmtfinsf1', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf',
       'firstflrsf', 'secondflrsf', 'lowqualfinsf', 'grlivarea',
       'bsmtfullbath', 'bsmthalfbath', 'fullbath', 'halfbath', 'bedroomabvgr',
       'kitchenabvgr', 'totrmsabvgrd', 'fireplaces', 'garagecars',
       'garagearea', 'wooddecksf', 'openporchsf', 'enclosedporch',
       'threessnporch', 'screenporch', 'poolarea', 'miscval', 'mosold',
       'yrsold', 'saleprice']]

In [218]:
new_df=pd.concat([cat_df_enc,num_df],axis=1)

In [219]:
#Split data into train and test sets.
X=new_df.drop('saleprice',axis=1)
Y=new_df['saleprice']
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
X_train.shape ,y_train.shape ,X_test.shape ,y_test.shape

((1168, 188), (1168,), (292, 188), (292,))

In [220]:
#Try OLS, lasso, ridge, and elastic net regression using the same model specification. 
#This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. 
#Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. 
#Which model is the best? Why?

In [221]:
# OLS
X_train=sm.add_constant(X_train)
X_test =sm.add_constant(X_test)
result=sm.OLS(y_train,X_train).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     67.75
Date:                Wed, 01 Sep 2021   Prob (F-statistic):               0.00
Time:                        02:54:15   Log-Likelihood:                -13314.
No. Observations:                1168   AIC:                         2.699e+04
Df Residuals:                     985   BIC:                         2.792e+04
Df Model:                         182                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.606e+06   9.92e+05     -1.620      0.106   -3.55e+06     3.4e+05
mszoning_FV            2.716e+04   1.29e+04      2.107      0.035    1859.161    5.25e+04
mszoning_RH            3.085e+04    1.3e+04      2.375      0.018    5364.187    5.63e+04
mszoning_RL            3.031e+04   1.08e+04      2.813      0.005    9163.893    5.15e+04
mszoning_RM            2.916e+04   9948.916      2.931      0.003    9633.431    4.87e+04
street_Pave            2.827e+04   1.36e+04      2.077      0.038    1561.080     5.5e+04
lotshape_IR2           4578.2691   5007.780      0.914      0.361   -5248.874    1.44e+04
lotshape_IR3          -2627.3651   1.02e+04     -0.258      0.797   -2.26e+04    1.74e+04
lotshape_Reg           1066.9831   1818.145      0.587      0.557   -2500.900    4634.866
landcontour_HLS        1.845e+04   5844.452      3.156      0.002    6976.250    2.99e+04
landcontour_Low        3088.2118   7300.310      0.423      0.672   -1.12e+04    1.74e+04
landcontour_Lvl        8699.5594   4152.797      2.095      0.036     550.213    1.68e+04
utilities_NoSeWa       1.449e-05   6.74e-06      2.149      0.032    1.26e-06    2.77e-05
lotconfig_CulDSac      4655.1349   3662.224      1.271      0.204   -2531.523    1.18e+04
lotconfig_FR2         -2557.2523   4607.441     -0.555      0.579   -1.16e+04    6484.276
lotconfig_FR3         -2.338e+04   1.51e+04     -1.544      0.123   -5.31e+04    6334.711
lotconfig_Inside       -990.7724   2007.402     -0.494      0.622   -4930.048    2948.503
landslope_Mod          7454.1708   4536.870      1.643      0.101   -1448.871    1.64e+04
landslope_Sev         -1.126e+04   1.36e+04     -0.830      0.407   -3.79e+04    1.54e+04
neighborhood_Blueste   -241.6478   1.95e+04     -0.012      0.990   -3.84e+04    3.79e+04
neighborhood_BrDale    1.441e+04   1.24e+04      1.162      0.246   -9933.978    3.88e+04
neighborhood_BrkSide  -1521.2419   1.01e+04     -0.150      0.881   -2.14e+04    1.84e+04
neighborhood_ClearCr  -1.924e+04   1.05e+04     -1.830      0.068   -3.99e+04    1391.783
neighborhood_CollgCr  -1.229e+04   7957.817     -1.545      0.123   -2.79e+04    3321.808
neighborhood_Crawfor   7539.5673   9383.182      0.804      0.422   -1.09e+04     2.6e+04
neighborhood_Edwards  -1.896e+04   8741.253     -2.168      0.030   -3.61e+04   -1801.603
neighborhood_Gilbert  -1.497e+04   8477.988     -1.765      0.078   -3.16e+04    1671.616
neighborhood_IDOTRR   -8866.4709   1.16e+04     -0.763      0.446   -3.17e+04    1.39e+04
neighborhood_MeadowV   5152.9801   1.25e+04      0.411      0.681   -1.94e+04    2.97e+04
neighborhood_Mitchel  -2.265e+04   8979.169     -2.522      0.012   -4.03e+04   -5025.863
neighborhood_NAmes    -1.571e+04   8587.619     -1.829      0.068   -3.26e+04    1143.210
neighborhood_NPkVill   

In [222]:
#Predict the house prices in the test set, 
#evaluate the performance of your model using the metrics previously mentioned in this lesson.

In [223]:
X_train.shape ,y_train.shape ,X_test.shape ,y_test.shape

((1168, 189), (1168,), (292, 189), (292,))

In [224]:
y_predict=result.predict(X_test)
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse,rmse

print("-----OLS-----")
print("R-squared of the model on the training set is: {}".format(result.rsquared))
print("-----Test set statistics-----")
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_pred_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_pred_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_pred_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100))



-----OLS-----
R-squared of the model on the training set is: 0.9260266349544116
-----Test set statistics-----
Mean absolute error of the prediction is: 82697.96139112668
Mean squared error of the prediction is: 12360746184.291525
Root mean squared error of the prediction is: 111178.89271031406
Mean absolute percentage error of the prediction is: 49.977142658220956


In [225]:
# Ridge
alphas = [np.power(10.0,p) for p in np.arange(-10,50,1)]

from sklearn.linear_model import RidgeCV
ridgereg=RidgeCV(alphas=alphas,cv=5)
ridgereg.fit(X_train,y_train)
y_pred_train=ridgereg.predict(X_train)
y_pred_test=ridgereg.predict(X_test)

print("-----Ridge-----")
print("Best alpha value is: {}".format(ridgereg.alpha_))
print("R-squared of the model on the training set is: {}".format(ridgereg.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgereg.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_pred_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_pred_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_pred_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100))



-----Ridge-----
Best alpha value is: 10.0
R-squared of the model on the training set is: 0.8854347675135468
-----Test set statistics-----
R-squared of the model on the test set is: 0.8370975658862468
Mean absolute error of the prediction is: 18256.51244384751
Mean squared error of the prediction is: 1028922580.4062886
Root mean squared error of the prediction is: 32076.82310339178
Mean absolute percentage error of the prediction is: 10.145537201092592


In [226]:
# Lasso

alphas = [np.power(10.0,p) for p in np.arange(-10,50,1)]

from sklearn.linear_model import LassoCV

lassoregr = LassoCV(alphas=alphas, cv=5) 
lassoregr.fit(X_train, y_train)

# You are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("-----Lasso-----")
print("Best alpha value is: {}".format(lassoregr.alpha_))
print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))



-----Lasso-----
Best alpha value is: 100.0
R-squared of the model on the training set is: 0.8862867848760203
-----Test set statistics-----
R-squared of the model on the test set is: 0.8405389494658451
Mean absolute error of the prediction is: 18050.428031774518
Mean squared error of the prediction is: 1007186150.9161346
Root mean squared error of the prediction is: 31736.19622633019
Mean absolute percentage error of the prediction is: 9.90617789934565


In [227]:
# Elastic net 

from sklearn.linear_model import ElasticNetCV

elasticregr = ElasticNetCV(alphas=alphas,cv=5) 
elasticregr.fit(X_train, y_train)

# You are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("-----Elastic Net-----")
print("Best alpha value is: {}".format(elasticregr.alpha_))
print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))





-----Elastic Net-----
Best alpha value is: 0.01
R-squared of the model on the training set is: 0.890544720604936
-----Test set statistics-----
R-squared of the model on the test set is: 0.8364425485307455
Mean absolute error of the prediction is: 18601.74250403523
Mean squared error of the prediction is: 1033059793.8942266
Root mean squared error of the prediction is: 32141.24754725968
Mean absolute percentage error of the prediction is: 10.41411803640497
